This tutorial will use torchtext and spacy for preprocessing and creating data for pytorch language models. Here we will use, IMDB movie review dataset as an example.

In [1]:
import torch
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, LabelField
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import spacy
from sklearn.model_selection import train_test_split

First, let us load the data, convert the label from String to Numeric values

In [33]:
pth = '../DataRepo/IMDB_Movie/'
fil = 'imdb_master.csv'
encoder = LabelEncoder()
df = pd.read_csv(pth+fil, encoding = "ISO-8859-1")
df['label'] = encoder.fit_transform(df['label'])
# select necessary columns
# df = df[['review','label']]
df.head(5)

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,0,0_2.txt
1,1,test,This is an example of why the majority of acti...,0,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",0,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,0,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,0,10003_3.txt


In [5]:
'''first step is to create a field object using torchtext, that says what are we going to do with each column of
the data''' 
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [6]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 180k/84.1M [00:00<00:47, 1.78MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:04<00:00, 18.7MB/s]


If you want to load your own data, and use torchtext and spacy libraries do as follows

In [34]:
# split data into test, validation and train
train = df[df['type'] == 'train']
test = df[df['type'] == 'test']
# select necessary columns and discard the rest
s_clm = ['review','label']
train,valid = train_test_split(df,train_size=0.8)
train,valid,test = train[s_clm],valid[s_clm],test[s_clm]
print('length of train data:{}'.format(len(train)))
print('length of validation data:{}'.format(len(valid)))
print('length of validation data:{}'.format(len(test)))

length of train data:80000
length of validation data:20000
length of validation data:25000


In [ ]:
# load the spacy english model, and return the tokenized text
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    # load the spacy language model, NOTE make sure to download the model in terminal using: python -m spacy download <language name>
    spacy_en = spacy.load('en')
    # tokenize text
    return [tok.text for tok in spacy_en.tokenizer(text)]

txt = Field(sequential=True, tokenize=tokenize_en, lower=True)
label = LabelField(sequential=True, tokenize=SimpleTok, init_token = '<sos>', eos_token = '<eos>', lower=True)
# first you  have to say how the data is structured 
d_format = [('review',src_txt),('label',dst_txt)]
